# Python-1, Лекция 11

Лектор: Хайбулин Даниэль

Подготовил материал: Лущ Иван

Итак, сегодня мы поговорим про простанства имен и декораторы

### [Namespaces](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)

Предыдущее занятие о функциях и классах дало интуитивное понимание локальных и глобальных переменных. Сегодня формализуем понятие "пространство имен" и механизмы разрешения имен в Python.

Под **пространством имен** (`namespace`) будем понимать отображение имя -> объект.

**Область видимости** (`scope`) — это текстовая область программы на Python, в пределах которой пространство имён доступно напрямую. "Доступно напрямую" означает, что обращение к имени пытается найти это имя в данном пространстве имён.


In [ ]:
# начало scope
# global namespace
x = "модуль"


def f() -> None:
    # начало scope
    x = "локальная"
    # достали x из local namespace
    print("в f:", x)
    # конец scope


# f создает новый local namespace при вызове
f()
# конец scope

Блоки `if`/`for`/`while`/`with` новых областей видимости не создают; функция (и `lambda`) при вызове — создает; `comprehension`/генераторные выражения в `Python 3` создают свой маленький локальный `scope` для переменной цикла.

Что будет?

In [ ]:
funcs = [lambda: i for i in range(3)]
print([f() for f in funcs])

lambda захватывает не значение i, а саму переменную i (ячейку) из enclosing‑scope comprehension.

<div style="
    background-color: #FFBA00;
    padding: 15px;
    border-left: 5px solid #ffcc00;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        Лучше не использовать lambda, обсуждали в конце лекции "Ссылки. Изменяемость."
    </span>
</div>

Python ищет имя по порядку `LEGB`: сначала в текущей локальной области (`L`, `Local`), потом во внешних функциях (`E`, `Enclosing`), потом в модуле (`G`, `Global`), потом во встроенных именах (`B`, `Builtins`).

![](legb.png)


- `Global`: содержит `funcs`
- `Enclosing` (охватывающее для `lambda`): каждое `list‑comprehension` в Python исполняется в своём локальном `scope`; переменная `i` — локальная именно для этого скрытого "функционального" `scope comprehension`.
- `Local` (у `lambda`): появляется только при вызове `lambda`; внутри тела `lambda` нет собственного `i`, поэтому поиск идёт во внешнее (`enclosing`) пространство.
- `Builtins`: здесь находятся `print` и `range`

Фикс через аргумент по умолчанию:

In [ ]:
funcs = [lambda i=i: i for i in range(3)]
print([f() for f in funcs])

Мы начали с необычного примера, чтобы интуитивно увидеть: имя ищется по цепочке областей видимости, а замыкания захватывают не значение, а ячейку имени. Теперь разберём формально каждое звено этой цепочки.

`builtins namespace`:

- Набор "встроенных" имён: `print`, `range`, `len`, `int`, `str`, `Exception`, `True`/`False`/`None`, ...
- Cоздаётся при инициализации интерпретатора; один на интерпретатор.
- Как используется: последнее звено в `LEGB`; имена доступны без импорта.

In [ ]:
print(dir(__builtins__))

In [ ]:
import builtins

builtins.print(dir(builtins))

`global namespace`:
- Что это: словарь `globals()` конкретного модуля (включая `__name__`, `__builtins__`, ваши функции/классы/переменные).
- Когда создаётся: при выполнении модуля (импорт или запуск).
- Время жизни: пока живёт модуль (обычно до завершения интерпретатора).


In [ ]:
globals()

`local namespace`:
- Что это: namespace текущего вызова функции/метода.
- Когда создаётся: при входе в функцию, уничтожается при выходе (если нет не являемся `enclosing`).

In [ ]:
def f(x: int) -> None:
    y = 1
    print(locals())


f(10)

In [ ]:
def f(arg: int) -> None:
    print(locals())
    print(locals() == globals())


f(1)

`enclosing namespace`:
- Что это: пространство имён внешней функции (и вложенных уровней), доступное для внутренних функций.
- Когда создаётся: при вызове внешней функции (или при выполнении `comprehension`).
- Время жизни: пока есть активные ссылки.


In [ ]:
from collections.abc import Callable


def make_counter() -> Callable[[], int]:
    count = 0

    def step() -> int:
        nonlocal count  # связываем имя из enclosing функции
        count += 1
        return count

    return step

In [ ]:
from collections.abc import Callable


x = 0


def outer() -> Callable[[], int]:
    x = 100

    def inner() -> int:
        global x  # это имя теперь относится к глобальному x, а не к outer.x
        x += 1
        return x

    return inner()


print(outer())
print(x)

- `global x` — связываем с именем из глобального пространства модуля.
- `nonlocal x` — связываем с именем во внешней функции.

In [ ]:
funcs = [lambda i=i: i for i in range(3)]  # копируем i из enclosing comprehension
print([f() for f in funcs])

Почему в Python стоит писать функцию `main`:

In [ ]:
def main() -> int:
    ...
    return 0


if __name__ == "__main__":
    raise SystemExit(main())


- `def main():` определение функции `main` — в ней размещается «стартовая» логика.
- `main()`: обычный вызов функции.
- `__name__`: специальная переменная модуля.
    - При прямом запуске файла: `__name__ == "__main__"`.
    - При импорте как модуля: `__name__ == "<имя_модуля>"`.

    `if __name__ == "__main__"`: `main()` — блок выполняется только при прямом запуске файла.

Более простая форма:

In [ ]:
def main() -> None:
    ...


if __name__ == "__main__":
    main()

Почему это полезно:

- Безопасный импорт: при импорте модуля код из `main` не выполнится.
- Чистый `namespace`: временные переменные живут в локальной области видимости `main`, а не засоряют глобальное пространство модуля (`if` не создаёт отдельного `scope`).
- Тестируемость и переиспользование: `main` можно вызывать из тестов и других модулей.
- Код возврата: удобно возвращать `int` и завершать процесс через `SystemExit`.

**Замыкание** - это любая функция, которая использует переменную, определенную в области видимости (`scope`), которая является внешней по отношению к этой функции, и доступна внутри функции при вызове из области, в которой эта переменная не определена.

In [ ]:
from collections.abc import Callable


def make_adder(x: int) -> Callable[[int], int]:
    def adder(y: int) -> int:
        return x + y

    return adder

In [ ]:
add_two = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

Представьте, что перед вами стоит задача реализовать механизм пометки функций как устаревших (`deprecated`) и применить его к функции `make_adder`. Требуется реализовать функцию `deprecated` для пометки устаревших функций и применить её к `make_adder`, при этом результирующая функция обязана сохранять исходное имя `make_adder`:

In [ ]:
import typing as tp
import warnings
from collections.abc import Callable


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


def deprecated(func: Callable[P, R]) -> Callable[P, R]:
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        warnings.warn(f"Warning: {func.__name__} is deprecated", DeprecationWarning)
        return func(*args, **kwargs)

    return wrapper

In [ ]:
import typing as tp
from collections.abc import Callable


def make_adder(x: int) -> Callable[[int], int]:
    """Returns a function that adds x"""

    def adder(y: int) -> int:
        return x + y

    return adder


make_adder = deprecated(make_adder)

In [ ]:
add_two = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

Мы сделали декоратор: он берёт функцию, добавляет к её вызову нужную логику и возвращает новую обёртку, которую можно вызывать так же, как оригинал.

### Декораторы

В Python предусмотрен синтаксический сахар для применения декораторов: конструкция `@deprecated`, размещённая непосредственно перед определением функции `make_adder`, эквивалентна присваиванию `make_adder = deprecated(make_adder)`, выполняемому сразу после оператора `def`.


In [ ]:
import typing as tp
import warnings
from collections.abc import Callable


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


def deprecated(func: Callable[P, R]) -> Callable[P, R]:
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        warnings.warn(f"Warning: {func.__name__} is deprecated", DeprecationWarning)
        return func(*args, **kwargs)

    return wrapper


@deprecated
def make_adder(x: int) -> Callable[[int], int]:
    """Returns a function that adds x"""

    def adder(y: int) -> int:
        return x + y

    return adder

В целом всё выполнено корректно; однако остаётся одно существенное замечание:

In [ ]:
print(make_adder.__name__)
print(make_adder.__doc__)

Для сохранения метаданных изначальной функции предлагается следущее решение:

In [ ]:
import typing as tp
import warnings
from collections.abc import Callable


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


def deprecated(func: Callable[P, R]) -> Callable[P, R]:
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        warnings.warn(f"Warning: {func.__name__} is deprecated", DeprecationWarning)
        return func(*args, **kwargs)

    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper


@deprecated
def make_adder(x: int) -> Callable[[int], int]:
    """Returns a function that adds x"""

    def adder(y: int) -> int:
        return x + y

    return adder


print(make_adder.__name__)
print(make_adder.__doc__)

Предложенное решение корректно; однако более идиоматичным считается использование `functools.wraps` для сохранения метаданных оригинальной функции:

In [ ]:
import functools
import typing as tp
import warnings
from collections.abc import Callable


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


def deprecated(func: Callable[P, R]) -> Callable[P, R]:
    @functools.wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        warnings.warn(f"Warning: {func.__name__} is deprecated", DeprecationWarning)
        return func(*args, **kwargs)

    return wrapper


@deprecated
def make_adder(x: int) -> Callable[[int], int]:
    """Returns a function that adds x"""

    def adder(y: int) -> int:
        return x + y

    return adder


print(make_adder.__name__)
print(make_adder.__doc__)

Хорошо, мы умеем создавать декораторы, а теперь реализуем параметризованный декоратор `deprecated`, принимающий аргумент даты начала устаревания (например, `since`) и включающий эту дату в предупреждение при каждом вызове помеченной функции, чтобы явно указывать, с какого момента функциональность считается устаревшей.

In [ ]:
import functools
import warnings
from collections.abc import Callable
from datetime import date, datetime


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


def deprecated(
    *, since: date | datetime | None = None
) -> Callable[[Callable[P, R]], Callable[P, R]]:
    def decorator(func: Callable[P, R]) -> Callable[P, R]:
        @functools.wraps(func)
        def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
            parts: list[str] = [f"Function {func.__name__}() is deprecated."]
            if since is not None:
                stamp = (
                    since.date().isoformat()
                    if isinstance(since, datetime)
                    else since.isoformat()
                )
                parts.append(f"Since {stamp}.")
            warnings.warn(" ".join(parts), DeprecationWarning)
            return func(*args, **kwargs)

        return wrapper

    return decorator


# decorator = deprecated(since=date(2024, 9, 1))
# make_adder = decorator(make_adder)
@deprecated(since=date(2024, 9, 1))
def make_adder(x: int) -> Callable[[int], int]:
    """Returns a function that adds x"""

    def adder(y: int) -> int:
        return x + y

    return adder


add_two = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

Главная идея: вы превратили "простой декоратор" в "параметризованный декоратор". Для этого добавили один внешний уровень вложенности. Чтобы декоратор мог принимать аргументы (`since`). Внешняя функция `deprecated(...)` получает параметры и возвращает "настоящий" декоратор `decorator`, который уже умеет оборачивать функцию.

Помимо атрибутов у классов и их экземпляров, атрибуты есть и у функций ([PEP 232](https://peps.python.org/pep-0232)). Это позволяет хранить состояние прямо на объекте функции. Давайте реализуем декоратор `count_calls`, который считает количество вызовов помеченной функции. Состояние будем хранить в атрибуте функции (например, `func.calls`). 

In [ ]:
import functools
import typing as tp
from collections.abc import Callable


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


def count_calls(func: Callable[P, R]) -> Callable[P, R]:
    @functools.wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        wrapper.calls += 1
        return func(*args, **kwargs)

    wrapper.calls = 0
    return wrapper


@count_calls
def add(a: int, b: int) -> int:
    return a + b


add(1, 2)
add(3, 4)
print(add.calls)

Класс в Python становится вызываемым при определении метода `__call__`. Следовательно, его можно использовать как декоратор: достаточно реализовать `__init__` (для приёма декорируемой функции или параметров) и `__call__` (для перехвата и обработки вызова). Предлагается создать класс‑декоратор для кэширования: при первом вызове вычислять результат и сохранять его по ключу, сформированному из аргументов; при последующих вызовах возвращать сохранённое значение. Это позволяет ускорять функции без изменения их интерфейса.

In [ ]:
import random
import typing as tp
from collections import deque
from collections.abc import Callable


P = tp.ParamSpec("P")
R = tp.TypeVar("R")


class Memoized(tp.Generic[P, R]):
    def __init__(self, cache_size: int = 100) -> None:
        self.cache_size: int = cache_size
        self.call_args_queue: deque[int] = deque()
        self.call_args_to_result: dict[int, R] = {}

    def __call__(self, fn: Callable[P, R]) -> Callable[P, R]:
        def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
            memoization_key: int = self._convert_call_arguments_to_hash(args, kwargs)
            if memoization_key not in self.call_args_to_result:
                result: R = fn(*args, **kwargs)
                self._update_cache_key_with_value(memoization_key, result)
                self._evict_cache_if_necessary()
            return self.call_args_to_result[memoization_key]

        return wrapper

    def _update_cache_key_with_value(self, key: int, value: R) -> None:
        self.call_args_to_result[key] = value
        self.call_args_queue.append(key)

    def _evict_cache_if_necessary(self) -> None:
        if len(self.call_args_queue) > self.cache_size:
            oldest_key: int = self.call_args_queue.popleft()
            del self.call_args_to_result[oldest_key]

    @staticmethod
    def _convert_call_arguments_to_hash(
        args: tuple[tp.Any, ...],
        kwargs: dict[str, tp.Any],
    ) -> int:
        return hash(str(args) + str(kwargs))


@Memoized(cache_size=5)
def get_not_so_random_number_with_max(max_value: float) -> float:
    return random.random() * max_value


In [ ]:
print(get_not_so_random_number_with_max(100))
print(get_not_so_random_number_with_max(100))

In [ ]:
print(get_not_so_random_number_with_max(1))
print(get_not_so_random_number_with_max(2))
print(get_not_so_random_number_with_max(3))
print(get_not_so_random_number_with_max(4))
print(get_not_so_random_number_with_max(5))

In [ ]:
print(get_not_so_random_number_with_max(100))

Можно сочетать несколько декораторов: Python применяет их снизу вверх. Запись
```python
@A
@B
def f(...):
```
эквивалентна `f = A(B(f))`. Порядок важен: где считать вызовы и где показывать предупреждение, зависит от расположения декораторов.

In [ ]:
@deprecated(since=date(2024, 9, 1))
@Memoized(cache_size=5)
@count_calls
def slow_add(a: int, b: int) -> int:
    return a + b

In [ ]:
slow_add(1, 2)

In [ ]:
slow_add(1, 2)

In [ ]:
slow_add(3, 4)

In [ ]:
slow_add.calls

In [ ]:
@count_calls
@deprecated(since=date(2024, 9, 1))
@Memoized(cache_size=5)
def slow_add(a: int, b: int) -> int:
    return a + b

In [ ]:
slow_add(3, 4)

In [ ]:
slow_add(3, 4)

In [ ]:
slow_add.calls

In [ ]:
slow_add(3, 5)

In [ ]:
slow_add.calls

In [ ]:
@count_calls
@Memoized(cache_size=5)
@deprecated(since=date(2024, 9, 1))
def slow_add(a: int, b: int) -> int:
    return a + b

In [ ]:
slow_add(3, 5)

In [ ]:
slow_add(3, 5)